In [1]:
%%configure -f
{ 
    "driverMemory": "1000M",
    "numExecutors": 1,
    "executorCores": 2,
    "jars" : ["https://repo1.maven.org/maven2/ai/rapids/cudf/0.8/cudf-0.8-cuda10.jar", 
      "https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark/0.90.1-Beta/xgboost4j-spark-0.90.1-Beta-cuda10.jar", 
      "https://repo1.maven.org/maven2/ai/rapids/xgboost4j/0.90.1-Beta/xgboost4j-0.90.1-Beta-cuda10.jar"]
}

In [2]:
%%info

In [3]:
// Databricks notebook source
import ml.dmlc.xgboost4j.scala.spark.{XGBoostClassifier, XGBoostClassificationModel}
import ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataReader
import ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.types.{DoubleType, IntegerType, StructField, StructType}


// COMMAND ----------

// val trainPath = "s3://sagemaker-gpu-xgboost/mortgage/csv/train/"
// val evalPath  = "s3://sagemaker-gpu-xgboost/mortgage/csv/test/"

val trainPath = "file:///root/mortgage/csv/train/"
val evalPath  = "file:///root/mortgage/csv/test/"


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,None,spark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import ml.dmlc.xgboost4j.scala.spark.{XGBoostClassifier, XGBoostClassificationModel}
import ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataReader
import ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.types.{DoubleType, IntegerType, StructField, StructType}
trainPath: String = file:///root/mortgage/csv/train/
evalPath: String = file:///root/mortgage/csv/test/


In [4]:

// COMMAND ----------

sc.listJars.foreach(println)

// COMMAND ----------

val spark = SparkSession.builder.appName("mortgage-gpu").getOrCreate

// COMMAND ----------

 val dataReader = new GpuDataReader(spark)

// COMMAND ----------

val labelColName = "delinquency_12"

val schema = StructType(List(
    StructField("orig_channel", DoubleType),
    StructField("first_home_buyer", DoubleType),
    StructField("loan_purpose", DoubleType),
    StructField("property_type", DoubleType),
    StructField("occupancy_status", DoubleType),
    StructField("property_state", DoubleType),
    StructField("product_type", DoubleType),
    StructField("relocation_mortgage_indicator", DoubleType),
    StructField("seller_name", DoubleType),
    StructField("mod_flag", DoubleType),
    StructField("orig_interest_rate", DoubleType),
    StructField("orig_upb", IntegerType),
    StructField("orig_loan_term", IntegerType),
    StructField("orig_ltv", DoubleType),
    StructField("orig_cltv", DoubleType),
    StructField("num_borrowers", DoubleType),
    StructField("dti", DoubleType),
    StructField("borrower_credit_score", DoubleType),
    StructField("num_units", IntegerType),
    StructField("zip", IntegerType),
    StructField("mortgage_insurance_percent", DoubleType),
    StructField("current_loan_delinquency_status", IntegerType),
    StructField("current_actual_upb", DoubleType),
    StructField("interest_rate", DoubleType),
    StructField("loan_age", DoubleType),
    StructField("msa", DoubleType),
    StructField("non_interest_bearing_upb", DoubleType),
    StructField(labelColName, IntegerType)))

 val commParamMap = Map(
    "eta" -> 0.1,
    "gamma" -> 0.1,
    "missing" -> 0.0,
    "max_depth" -> 10,
    "max_leaves" -> 256,
    "grow_policy" -> "depthwise",
    "min_child_weight" -> 30,
    "lambda" -> 1,
    "scale_pos_weight" -> 2,
    "subsample" -> 1,
    "nthread" -> 1,
    "num_round" -> 100,
    "num_workers" -> 1,
    "tree_method" -> "gpu_hist")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

spark://spark:35981/jars/livy-repl_2.11-0.6.0-incubating.jar
https://repo1.maven.org/maven2/ai/rapids/cudf/0.8/cudf-0.8-cuda10.jar
spark://spark:35981/jars/livy-core_2.11-0.6.0-incubating.jar
spark://spark:35981/jars/livy-rsc-0.6.0-incubating.jar
https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark/0.90.1-Beta/xgboost4j-spark-0.90.1-Beta-cuda10.jar
https://repo1.maven.org/maven2/ai/rapids/xgboost4j/0.90.1-Beta/xgboost4j-0.90.1-Beta-cuda10.jar
spark://spark:35981/jars/livy-api-0.6.0-incubating.jar
spark://spark:35981/jars/netty-all-4.0.37.Final.jar
spark://spark:35981/jars/commons-codec-1.9.jar
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1f36fa5d
dataReader: ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataReader = ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataReader@5afbb607
labelColName: String = delinquency_12
schema: org.apache.spark.sql.types.StructType = StructType(StructField(orig_channel,DoubleType,true), StructField(first_home_buyer,DoubleType,true

In [5]:
// COMMAND ----------

var (trainSet, evalSet) = {
  dataReader.option("header", true).schema(schema)
  (dataReader.csv(trainPath), dataReader.csv(evalPath))}

val featureNames = schema.filter(_.name != labelColName).map(_.name)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trainSet: ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset = ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset@2fff8496
evalSet: ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset = ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset@798ad87f
featureNames: Seq[String] = List(orig_channel, first_home_buyer, loan_purpose, property_type, occupancy_status, property_state, product_type, relocation_mortgage_indicator, seller_name, mod_flag, orig_interest_rate, orig_upb, orig_loan_term, orig_ltv, orig_cltv, num_borrowers, dti, borrower_credit_score, num_units, zip, mortgage_insurance_percent, current_loan_delinquency_status, current_actual_upb, interest_rate, loan_age, msa, non_interest_bearing_upb)


In [6]:
// COMMAND ----------


object Benchmark {
  def time[R](phase: String)(block: => R): (R, Float) = {
    val t0 = System.currentTimeMillis
    val result = block // call-by-name
    val t1 = System.currentTimeMillis
    println("==> Benchmark: Elapsed time for [" + phase + "]: " + ((t1 - t0).toFloat / 1000) + "s")
    (result, (t1 - t0).toFloat / 1000)
  }
}


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object Benchmark


In [7]:
// COMMAND ----------

val modelPath = "/tmp/model"
val xgbClassifier = new XGBoostClassifier(commParamMap).setLabelCol(labelColName).setFeaturesCols(featureNames)
println("\n------ Training ------")

val (model, _) = Benchmark.time("train") {
        xgbClassifier.fit(trainSet)
}
// Save model if modelPath exists
model.write.overwrite().save(modelPath)
val xgbClassificationModel = model


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelPath: String = /tmp/model
xgbClassifier: ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier = xgbc_6c4ce4a690f8

------ Training ------
==> Benchmark: Elapsed time for [train]: 17.737s
model: ml.dmlc.xgboost4j.scala.spark.XGBoostClassificationModel = xgbc_6c4ce4a690f8
xgbClassificationModel: ml.dmlc.xgboost4j.scala.spark.XGBoostClassificationModel = xgbc_6c4ce4a690f8


In [8]:
// COMMAND ----------

println("\n------ Transforming ------")
val (results, _) = Benchmark.time("transform") {
  xgbClassificationModel.transform(evalSet)
}
// results.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


------ Transforming ------
==> Benchmark: Elapsed time for [transform]: 0.48s
results: org.apache.spark.sql.DataFrame = [orig_channel: float, first_home_buyer: float ... 29 more fields]


In [9]:
// COMMAND ----------

println("\n------Accuracy of Evaluation------")
val evaluator = new MulticlassClassificationEvaluator().setLabelCol(labelColName)
val accuracy = evaluator.evaluate(results)
println(accuracy)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


------Accuracy of Evaluation------
evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_60310ef193bc
accuracy: Double = 0.9875245818304412
0.9875245818304412
